In [42]:
import QuantLib as ql
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from random import sample
import sklearn.neural_network as sknn
import matplotlib.pyplot as plt
import time

In [99]:
def MakeUniformRandParams(ArrayMin,ArrayMax,nRand):
    if (len(ArrayMin)!=len(ArrayMax)):
        raise Exception('ArrayMax and ArrayMin must have the same size')
    else:
        nParams = len(ArrayMin)
        rand_seq = ql.UniformRandomSequenceGenerator(nParams*nRand,ql.UniformRandomGenerator())
        X = rand_seq.nextSequence().value()
        X = [ArrayMin[j%nParams]+(ArrayMax[j%nParams]-ArrayMin[j%nParams])*X[j] for j in range(nParams*nRand)]
        res = np.reshape(X,(nRand,nParams))
        return res

def RemoveOutFellerCondition(X):
    k=0
    for (j in range(len(X))):
        if (2*X[k,0]*X[k,1]<X[k,3]**2):
            X.remove(X[k,:])
        
## TBC    
    
ArrayMin = [0.1, 0.1,  0.01, -0.999, 0.1]
ArrayMax = [1.5, 1.5,  0.75,  0.999, 1.5]
nRand = 500000
#start = time.time()
#res=MakeUniformRandParams(ArrayMin,ArrayMax,nRand)
#end = time.time()
#print('Total computation time = ' + str(round(end-start,2)) + 's')
X = [1,2,3,4]
X.remove(3)
print(X)

[1, 2, 4]


In [69]:
# GLOBAL CONFIG #
AsOfDate = ql.Date(15,10,2018)
ql.Settings.instance().evaluationDate = AsOfDate
DayCount = ql.Actual365Fixed()
Calendar = ql.UnitedStates()

In [45]:
# EUROPEAN OPTION CONFIG #
MaturityDate = ql.Date(15,10,2022)
TargetDate = ql.Date(13,10,2022)
strike = 100
option_type = ql.Option.Call
payoff = ql.PlainVanillaPayoff(option_type, strike)
exercise = ql.EuropeanExercise(MaturityDate)
EuropeanOption = ql.VanillaOption(payoff, exercise)

In [46]:
# MARKET DATA
r=ql.SimpleQuote(0.01)
r_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(r),DayCount)
q=ql.SimpleQuote(0.0)
q_ts = ql.FlatForward(0,Calendar,ql.QuoteHandle(q),DayCount)
s0=ql.SimpleQuote(100)

# BLACK SCHOLES ENGINE ONLY USED FOR COMPUTING IMPLIED VOLATILITIES
vol_bs = ql.SimpleQuote(0.10)
vol_bs_ts = ql.BlackConstantVol(0,Calendar,ql.QuoteHandle(vol_bs),DayCount)
BSProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(s0),ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                               ql.BlackVolTermStructureHandle(vol_bs_ts))

# HESTON ENGINE
v0 = 0.01
kappa=0.5
theta=0.01
sigma=0.0001
rho=-0.5
HestonParams = [theta,kappa,sigma,rho,v0]
relTolerance=0.01
maxEval=20000
HestonProcess = ql.HestonProcess(ql.YieldTermStructureHandle(r_ts),ql.YieldTermStructureHandle(q_ts),
                                 ql.QuoteHandle(s0),v0,kappa,theta,sigma,rho)
HestonModel = ql.HestonModel(HestonProcess)
HestonEngine = ql.AnalyticHestonEngine(HestonModel,relTolerance,maxEval)
EuropeanOption.setPricingEngine(HestonEngine)

In [47]:
# GENERATE DATAS
nRand = 200000
start = time.time()
MinParamsValues = [0.1, 0.1,  0.01, -0.999, 0.1]
MaxParamsValues = [1.5, 1.5,  0.75,  0.999, 1.5]
local_start = 0
local_end = 0
ArrParams = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nRand)
intermediary = time.time()

#ArrParams[1,:]
npv = np.zeros(nRand)
iv = np.zeros(nRand)
for i in range(nRand):
    HestonModel.setParams([ArrParams[i][k] for k in range(len(MinParamsValues))])
    npv[i] = EuropeanOption.NPV()
    iv[i] = EuropeanOption.impliedVolatility(npv[i],BSProcess,1.0e-4,200,1.0e-8,10)
end = time.time()
print('Random params simulation time = ' + str(round(intermediary-start,2)) + 's')
print('Prices computation time = ' + str(round(end-intermediary,2)) + 's')
print('Total computation time = ' + str(round(end-start,2)) + 's')

Random params simulation time = 0.14s
Prices computation time = 31.05s
Total computation time = 31.19s


In [48]:
def ann_layers_neurones(layer, neurone):
    hidden_layer_sizes = (neurone, )
    if layer == 1.0: return hidden_layer_sizes
    elif layer > 1.0:
        for i in range(layer-1):
            hidden_layer_sizes += (neurone,)
        return hidden_layer_sizes

nLayersMin = 3
nLayersMax = 5
nLayersStep = 1
nNeuronesMin = 30
nNeuronesMax = 60
nNeuronesStep = 5
hidden_layers_sizes_vect = []
i_ = 0
for i in range(nNeuronesMin,nNeuronesMax+1,nNeuronesStep):
    j_ = 0
    for j in range(nLayersMin,nLayersMax+1,nLayersStep):
        hidden_layers_sizes_vect.append(ann_layers_neurones(j,i))
        j_ += 1
    i_ += 1
print(hidden_layers_sizes_vect)

[(30, 30, 30), (30, 30, 30, 30), (30, 30, 30, 30, 30), (35, 35, 35), (35, 35, 35, 35), (35, 35, 35, 35, 35), (40, 40, 40), (40, 40, 40, 40), (40, 40, 40, 40, 40), (45, 45, 45), (45, 45, 45, 45), (45, 45, 45, 45, 45), (50, 50, 50), (50, 50, 50, 50), (50, 50, 50, 50, 50), (55, 55, 55), (55, 55, 55, 55), (55, 55, 55, 55, 55), (60, 60, 60), (60, 60, 60, 60), (60, 60, 60, 60, 60)]


In [49]:
start = time.time()
nMin = nRand
nMax = nRand
nStep = 500
vect_train_error = np.zeros(int((nMax-nMin)/nStep)+1)
vect_test_error = np.zeros(int((nMax-nMin)/nStep)+1)
i = 0
for n in range(nMin,nMax+1,nStep):
    index = sample(range(0,nRand),n)
    X=np.zeros((n,len(ArrParams[0])))
    for k in range(len(ArrParams[0])):
        X[:,k]=[ArrParams[j][k] for j in index]
    Y=[iv[j] for j in index]

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    # ANN FIT
    vect_train_error[i]=1000
    vect_test_error[i]=1000
    for current_archi in hidden_layers_sizes_vect:
        ann = sknn.MLPRegressor(hidden_layer_sizes=current_archi)
        ann.fit(X_train,Y_train)
        if (np.mean(abs(ann.predict(X_train)-Y_train))*100<vect_train_error[i]):
            vect_train_error[i]=np.mean(abs(ann.predict(X_train)-Y_train))*100
            vect_test_error[i]=np.mean(abs(ann.predict(X_test)-Y_test))*100
    i+=1
end = time.time()
print('Total computation time = ' + str(round(end-start,2)) + 's')

Total computation time = 281.95s


In [50]:
print(vect_train_error)
print(vect_test_error)
#plt.plot(range(nMin,nMax+1,nStep),vect_train_error,color='b')
#plt.plot(range(nMin,nMax+1,nStep),vect_test_error, color='r')
#plt.show()

[0.42243789]
[0.4285705]


In [51]:
nTest = 3000
X_test = MakeUniformRandParams(MinParamsValues,MaxParamsValues,nTest)
X_testAfterScale = scaler.transform(X_test)
iv = np.zeros(nTest)
err = np.zeros(nTest)
prediction = ann.predict(X_testAfterScale)
for i in range(nTest):
    HestonModel.setParams([X_test[i,k] for k in range(len(MinParamsValues))])
    iv[i] = EuropeanOption.impliedVolatility(EuropeanOption.NPV(),BSProcess,1.0e-4,200,1.0e-8,10)
    err[i]=abs(iv[i]-prediction[i])
stats.describe(err)

DescribeResult(nobs=3000, minmax=(3.6815891468755524e-07, 0.060570118918586824), mean=0.0047184862438876644, variance=1.4214250562972097e-05, skewness=2.378469056236739, kurtosis=19.338542371363665)

8.426932725704573